**2. Pre-processing the Wyscout data**

**A) The following tasks are taken into account in this notebook:**

1. The Wyscout data jsons (Competitions, Matches, Events and Players) are processed and stored in dataframes. T

2. The defence formation data (as obtained from 1_PLparser.ipynb) is merged with the Wyscout data, in such a way that each row of the new dataframe denotes a team for a particular match.

3. The resulting dataframe is further processed to normalise team and player names

4. Footedness and event tags information is added to the dataframe

5. Event coordinates are converted to the scale of x belongs to [0,104] and y belongs to [0,68]

**B) The following were the resulting pickle files of this notebook:**

**events_v2.pkl** - processed Wyscout events data with scaled coordinates

**match+def_lineup+footedness_ver2.pkl** - defence formation data with each row denoting a team for a particular match

**Note:**

1. All experiments have been run on Premier League data only

2. We understand that the role of center backs changes with change in number of players in the defence lineups.  For simplicity, our analysis includes two categories - lineups with four defenders and lineups with other than four defenders. The following is the formation nomenclature:


    a. For a defence lineup with 4 players:
            i. LB - Left Back
            ii. L_CB - Left Center Back
            iii. R_CB - Right Center Back
            iv. RB - Right Back
            
            
    b. For a defence lineup with 3 or 5 players:
            i. LWB - Left Wing Back (only for 5 defenders)
            ii. LCB - Left Center Back (for both 3 and 5 defenders)
            iii. CB - Center Back (for both 3 and 5 defenders)
            iv. RCB - Right Center Back (for both 3 and 5 defenders)
            v. RB - Right Wing Back (only for 5 defenders)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
import json
import operator
from matplotlib.ticker import FuncFormatter
from matplotlib.patches import Ellipse
import base64
from collections import defaultdict
import sys,os
import math
import random
import matplotlib.pylab as pyl
import itertools
import pickle
import swifter
import warnings
from unidecode import unidecode
from itertools import chain
from multiprocessing import  Pool

warnings.filterwarnings('ignore')

In [2]:
# pd.set_option('max_colwidth', 999)

In [3]:
pd.set_option('display.max_columns', 1000)
pd.set_option("display.max_rows", 3000)
warnings.filterwarnings('ignore')

# Competitions 

- area: it denotes the geographic area associated with the league as a sub-document, using the ISO 3166-1 specification (https://www.iso.org/iso-3166-country-codes.html);
- format: the format of the competition. All competitions for clubs have value "Domestic league". The competitions for national teams have value "International cup";
- name: the official name of the competition (e.g., Italian first division, Spanish first division, World Cup, etc.);
- type: the typology of the competition. It is "club" for the competitions for clubs and "international" for the competitions for national teams (World Cup 2018, European Cup 2016);
- wyId: the unique identifier of the competition, assigned by Wyscout.

In [4]:
comps = pd.read_json('../data/competitions/competitions.json')
comps = pd.DataFrame(comps)
comps

,name,wyId,format,area,type
0,Italian first division,524,Domestic league,"{'name': 'Italy', 'id': '380', 'alpha3code': '...",club
1,English first division,364,Domestic league,"{'name': 'England', 'id': '0', 'alpha3code': '...",club
2,Spanish first division,795,Domestic league,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
3,French first division,412,Domestic league,"{'name': 'France', 'id': '250', 'alpha3code': ...",club
4,German first division,426,Domestic league,"{'name': 'Germany', 'id': '276', 'alpha3code':...",club
5,European Championship,102,International cup,"{'name': '', 'id': 0, 'alpha3code': 'XEU', 'al...",international
6,World Cup,28,International cup,"{'name': '', 'id': 0, 'alpha3code': 'XWO', 'al...",international


# Matches 

- competitionId: the identifier of the competition to which the match belongs to. It is a integer and refers to the field "wyId" of the competition document;
- date and dateutc: the former specifies date and time when the match starts in explicit format (e.g., May 20, 2018 at 8:45:00 PM GMT+2), the latter contains the same information but in the compact format YYYY-MM-DD hh:mm:ss;
- duration: the duration of the match. It can be "Regular" (matches of regular duration of 90 minutes + stoppage time), "ExtraTime" (matches with supplementary times, as it may happen for matches in continental or international competitions), or "Penalities" (matches which end at penalty kicks, as it may happen for continental or international competitions);
- gameweek: the week of the league, starting from the beginning of the league;
- label: contains the name of the two clubs and the result of the match (e.g., "Lazio - Internazionale, 2 - 3");
- roundID: indicates the match-day of the competition to which the match belongs to. During a competition for soccer clubs, each of the participating clubs plays against each of the other clubs twice, once at home and once away. The matches are organized in match-days: all the matches in match-day i are played before the matches in match-day i + 1, even tough some matches can be anticipated or postponed to facilitate players and clubs participating in Continental or Intercontinental competitions. During a competition for national teams, the "roundID" indicates the stage of the competition (eliminatory round, round of 16, quarter finals, semifinals, final);
- seasonId: indicates the season of the match;
- status: it can be "Played" (the match has officially finished), "Cancelled" (the match has been canceled for some reason), "Postponed" (the match has been postponed and no new date and time is available yet) or "Suspended" (the match has been suspended and no new date and time is available yet);
- venue: the stadium where the match was held (e.g., "Stadio Olimpico");
- winner: the identifier of the team which won the game, or 0 if the match ended with a draw;
- wyId: the identifier of the match, assigned by Wyscout;
- teamsData: it contains several subfields describing information about each team that is playing that match: such as lineup, bench composition, list of substitutions, coach and scores:
- hasFormation: it has value 0 if no formation (lineups and benches) is present, and 1 otherwise;
- score: the number of goals scored by the team during the match (not counting penalties);
- scoreET: the number of goals scored by the team during the match, including the extra time (not counting penalties);
- scoreHT: the number of goals scored by the team during the first half of the match;
- scoreP: the total number of goals scored by the team after the penalties;
- side: the team side in the match (it can be "home" or "away");
- teamId: the identifier of the team;
- coachId: the identifier of the team's coach;
- bench: the list of the team's players that started the match in the bench and some basic statistics about their performance during the match (goals, own goals, cards);
- lineup: the list of the team's players in the starting lineup and some basic statistics about their performance during the match (goals, own goals, cards);
- substitutions: the list of team's substitutions during the match, describing the players involved and the minute of the substitution.

In [5]:
matches = pd.read_json('../data/matches/matches_England.json')
matches = pd.DataFrame(matches)
print(matches.shape)
display(matches)

(380, 14)


,status,roundId,gameweek,teamsData,seasonId,dateutc,winner,venue,wyId,label,date,referees,duration,competitionId
0,Played,4405654,38,"{'1646': {'scoreET': 0, 'coachId': 8880, 'side...",181150,2018-05-13 14:00:00,1659,Turf Moor,2500089,"Burnley - AFC Bournemouth, 1 - 2","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 385705, 'role': 'referee'}, {'r...",Regular,364
1,Played,4405654,38,"{'1628': {'scoreET': 0, 'coachId': 8357, 'side...",181150,2018-05-13 14:00:00,1628,Selhurst Park,2500090,"Crystal Palace - West Bromwich Albion, 2 - 0","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 381851, 'role': 'referee'}, {'r...",Regular,364
2,Played,4405654,38,"{'1609': {'scoreET': 0, 'coachId': 7845, 'side...",181150,2018-05-13 14:00:00,1609,The John Smith's Stadium,2500091,"Huddersfield Town - Arsenal, 0 - 1","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 384965, 'role': 'referee'}, {'r...",Regular,364
3,Played,4405654,38,"{'1651': {'scoreET': 0, 'coachId': 8093, 'side...",181150,2018-05-13 14:00:00,1612,Anfield,2500092,"Liverpool - Brighton & Hove Albion, 4 - 0","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 385704, 'role': 'referee'}, {'r...",Regular,364
4,Played,4405654,38,"{'1644': {'scoreET': 0, 'coachId': 93112, 'sid...",181150,2018-05-13 14:00:00,1611,Old Trafford,2500093,"Manchester United - Watford, 1 - 0","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 381853, 'role': 'referee'}, {'r...",Regular,364
5,Played,4405654,38,"{'1613': {'scoreET': 0, 'coachId': 210700, 'si...",181150,2018-05-13 14:00:00,1613,St. James' Park,2500094,"Newcastle United - Chelsea, 3 - 0","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 384888, 'role': 'referee'}, {'r...",Regular,364
6,Played,4405654,38,"{'1625': {'scoreET': 0, 'coachId': 267136, 'si...",181150,2018-05-13 14:00:00,1625,St. Mary's Stadium,2500095,"Southampton - Manchester City, 0 - 1","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 385911, 'role': 'referee'}, {'r...",Regular,364
7,Played,4405654,38,"{'10531': {'scoreET': 0, 'coachId': 32573, 'si...",181150,2018-05-13 14:00:00,1639,Liberty Stadium,2500096,"Swansea City - Stoke City, 1 - 2","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 378952, 'role': 'referee'}, {'r...",Regular,364
8,Played,4405654,38,"{'1631': {'scoreET': 0, 'coachId': 209010, 'si...",181150,2018-05-13 14:00:00,1624,Wembley Stadium,2500097,"Tottenham Hotspur - Leicester City, 5 - 4","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 378951, 'role': 'referee'}, {'r...",Regular,364
9,Played,4405654,38,"{'1623': {'scoreET': 0, 'coachId': 8541, 'side...",181150,2018-05-13 14:00:00,1633,London Stadium,2500098,"West Ham United - Everton, 3 - 1","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 408156, 'role': 'referee'}, {'r...",Regular,364


Sample 'TeamsData' row entry:

# Events

- eventId: the identifier of the event's type. Each eventId is associated with an event name (see next point);
- eventName: tteamIdhe name of the event's type. There are seven types of events: pass, foul, shot, duel, free kick, offside and touch;
- subEventId: the identifier of the subevent's type. Each subEventId is associated with a subevent name (see next point);
- subEventName: the name of the subevent's type. Each event type is associated with a different set of subevent types;
- tags: a list of event tags, each one describes additional information about the event (e.g., accurate). Each event type is associated with a different set of tags;
- eventSec: the time when the event occurs (in seconds since the beginning of the current half of the match);
- id: a unique identifier of the event;
- matchId: the identifier of the match the event refers to. The identifier refers to the field "wyId" in the match dataset;
- matchPeriod: the period of the match. It can be "1H" (first half of the match), "2H" (second half of the match), "E1" (first extra time), "E2" (second extra time) or "P" (penalties time);
- playerId: the identifier of the player who generated the event. The identifier refers to the field "wyId" in a player dataset;
- positions: the origin and destination positions associated with the event. Each position is a pair of coordinates (x, y). The x and y coordinates are always in the range [0, 100] and indicate the percentage of the field from the perspective of the attacking team. In particular, the value of the x coordinate indicates the event's nearness (in percentage) to the opponent's goal, while the value of the y coordinates indicates the event's nearness (in percentage) to the right side of the field;
- teamId: the identifier of the player's team. The identifier refers to the field "wyId" in the team dataset.

In [6]:
events = pd.read_json('../data/events/events_England.json')
events = pd.DataFrame(events)
print(events.shape)
display(events)

(643150, 12)


,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id
0,8,Simple pass,[{'id': 1801}],25413,"[{'y': 49, 'x': 49}, {'y': 78, 'x': 31}]",2499719,Pass,1609,1H,2.758649,85,177959171
1,8,High pass,[{'id': 1801}],370224,"[{'y': 78, 'x': 31}, {'y': 75, 'x': 51}]",2499719,Pass,1609,1H,4.946850,83,177959172
2,8,Head pass,[{'id': 1801}],3319,"[{'y': 75, 'x': 51}, {'y': 71, 'x': 35}]",2499719,Pass,1609,1H,6.542188,82,177959173
3,8,Head pass,[{'id': 1801}],120339,"[{'y': 71, 'x': 35}, {'y': 95, 'x': 41}]",2499719,Pass,1609,1H,8.143395,82,177959174
4,8,Simple pass,[{'id': 1801}],167145,"[{'y': 95, 'x': 41}, {'y': 88, 'x': 72}]",2499719,Pass,1609,1H,10.302366,85,177959175
...,...,...,...,...,...,...,...,...,...,...,...,...
643145,5,Ball out of the field,[],0,"[{'y': 32, 'x': 0}, {'y': 100, 'x': 100}]",2500098,Interruption,1623,2H,2796.732525,50,251596409
643146,3,Corner,"[{'id': 302}, {'id': 801}, {'id': 1801}]",70965,"[{'y': 100, 'x': 100}, {'y': 47, 'x': 88}]",2500098,Free Kick,1633,2H,2829.821084,30,251596232
643147,1,Air duel,"[{'id': 701}, {'id': 1802}]",7919,"[{'y': 53, 'x': 12}, {'y': 50, 'x': 14}]",2500098,Duel,1623,2H,2831.211419,10,251596410
643148,1,Air duel,"[{'id': 703}, {'id': 1801}]",8005,"[{'y': 47, 'x': 88}, {'y': 50, 'x': 86}]",2500098,Duel,1633,2H,2832.434399,10,251596234


In [7]:
print(len(events.matchId.unique()))
print(matches.shape)

380
(380, 14)


In [8]:
events.matchId.value_counts().mean()

1692.5

Approximately, every 3.19 seconds, an event activity is registered

# Player 

- birthArea: geographic information about the player's birth area;
- birthDate: the birth date of the player, in the format "YYYY-MM-DD";
- currentNationalTeamId: the identifier of the national team where the players currently plays;
- currentTeamId: the identifier of the team where the player plays for. The identifier refers to the field "wyId" in a team document;
- firstName: the first name of the player;
- lastName: the last name of the player;
- foot: the preferred foot of the player;
- height: the height of the player (in centimeters);
- middleName: the middle name (if any) of the player;
- passportArea: the geographic area associated with the player's current passport;
- role: the main role of the player. It is a subdocument containing the role's name and two abbreviations of it;
- shortName2: the short name of the player;
- weight: the weight of the player (in kilograms);
- wyId: the identifier of the player, assigned by Wyscout.

In [9]:
players = pd.read_json('../data/players/players.json')
players = pd.DataFrame(players)
print(players.shape)
display(players)

(3603, 14)


,passportArea,weight,firstName,middleName,lastName,currentTeamId,birthDate,height,role,birthArea,wyId,foot,shortName,currentNationalTeamId
0,"{'name': 'Turkey', 'id': '792', 'alpha3code': ...",78,Harun,,Tekin,4502,1989-06-17,187,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'Turkey', 'id': '792', 'alpha3code': ...",32777,right,H. Tekin,4687
1,"{'name': 'Senegal', 'id': '686', 'alpha3code':...",73,Malang,,Sarr,3775,1999-01-23,182,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'France', 'id': '250', 'alpha3code': ...",393228,left,M. Sarr,4423
2,"{'name': 'France', 'id': '250', 'alpha3code': ...",72,Over,,Mandanda,3772,1998-10-26,176,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'France', 'id': '250', 'alpha3code': ...",393230,,O. Mandanda,null
3,"{'name': 'Senegal', 'id': '686', 'alpha3code':...",82,Alfred John Momar,,N'Diaye,683,1990-03-06,187,"{'code2': 'MD', 'code3': 'MID', 'name': 'Midfi...","{'name': 'France', 'id': '250', 'alpha3code': ...",32793,right,A. N'Diaye,19314
4,"{'name': 'France', 'id': '250', 'alpha3code': ...",84,Ibrahima,,Konat\u00e9,2975,1999-05-25,192,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'France', 'id': '250', 'alpha3code': ...",393247,right,I. Konat\u00e9,null
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3598,"{'name': 'Tunisia', 'id': 788, 'alpha3code': '...",72,Ali,,Ma\u00e2loul,16041,1990-01-01,175,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'Tunisia', 'id': 788, 'alpha3code': '...",120839,left,A. Ma\u00e2loul,null
3599,"{'name': 'Peru', 'id': 604, 'alpha3code': 'PER...",76,Carlos Alberto,,C\u00e1ceda Oyaguez,15591,1991-09-27,183,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'Peru', 'id': 604, 'alpha3code': 'PER...",114736,right,C. C\u00e1ceda,null
3600,"{'name': 'Peru', 'id': 604, 'alpha3code': 'PER...",78,Miguel Gianpierre,,Araujo Blanco,12072,1994-10-24,179,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'Peru', 'id': 604, 'alpha3code': 'PER...",114908,right,M. Araujo,null
3601,"{'name': 'Morocco', 'id': 504, 'alpha3code': '...",70,Ahmed Reda,,Tagnaouti,16183,1996-04-05,182,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'Morocco', 'id': 504, 'alpha3code': '...",285583,right,A. Tagnaouti,null


# Events + Player DF 

In [10]:
events_com = pd.merge(right=players, left=events, right_on='wyId', left_on='playerId', how='left')
print(events_com.shape)
display(events_com)

(643150, 26)


,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,passportArea,weight,firstName,middleName,lastName,currentTeamId,birthDate,height,role,birthArea,wyId,foot,shortName,currentNationalTeamId
0,8,Simple pass,[{'id': 1801}],25413,"[{'y': 49, 'x': 49}, {'y': 78, 'x': 31}]",2499719,Pass,1609,1H,2.758649,85,177959171,"{'name': 'Guadeloupe', 'id': 312, 'alpha3code'...",73.0,Alexandre,,Lacazette,1609,1991-05-28,175.0,"{'code2': 'FW', 'code3': 'FWD', 'name': 'Forwa...","{'name': 'France', 'id': 250, 'alpha3code': 'F...",25413.0,right,A. Lacazette,null
1,8,High pass,[{'id': 1801}],370224,"[{'y': 78, 'x': 31}, {'y': 75, 'x': 51}]",2499719,Pass,1609,1H,4.946850,83,177959172,"{'name': 'England', 'id': 0, 'alpha3code': 'XE...",75.0,Rob,,Holding,1609,1995-09-20,189.0,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'England', 'id': 0, 'alpha3code': 'XE...",370224.0,right,R. Holding,null
2,8,Head pass,[{'id': 1801}],3319,"[{'y': 75, 'x': 51}, {'y': 71, 'x': 35}]",2499719,Pass,1609,1H,6.542188,82,177959173,"{'name': 'Germany', 'id': '276', 'alpha3code':...",76.0,Mesut,,\u00d6zil,1609,1988-10-15,180.0,"{'code2': 'MD', 'code3': 'MID', 'name': 'Midfi...","{'name': 'Germany', 'id': '276', 'alpha3code':...",3319.0,left,M. \u00d6zil,3148
3,8,Head pass,[{'id': 1801}],120339,"[{'y': 71, 'x': 35}, {'y': 95, 'x': 41}]",2499719,Pass,1609,1H,8.143395,82,177959174,"{'name': 'Egypt', 'id': '818', 'alpha3code': '...",70.0,Mohamed Naser,,Elsayed Elneny,1609,1992-07-11,180.0,"{'code2': 'MD', 'code3': 'MID', 'name': 'Midfi...","{'name': 'Egypt', 'id': '818', 'alpha3code': '...",120339.0,right,Mohamed Elneny,16129
4,8,Simple pass,[{'id': 1801}],167145,"[{'y': 95, 'x': 41}, {'y': 88, 'x': 72}]",2499719,Pass,1609,1H,10.302366,85,177959175,"{'name': 'Spain', 'id': 724, 'alpha3code': 'ES...",74.0,H\u00e9ctor,,Beller\u00edn Moruno,1609,1995-03-19,178.0,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'Spain', 'id': 724, 'alpha3code': 'ES...",167145.0,right,Beller\u00edn,null
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
643145,5,Ball out of the field,[],0,"[{'y': 32, 'x': 0}, {'y': 100, 'x': 100}]",2500098,Interruption,1623,2H,2796.732525,50,251596409,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
643146,3,Corner,"[{'id': 302}, {'id': 801}, {'id': 1801}]",70965,"[{'y': 100, 'x': 100}, {'y': 47, 'x': 88}]",2500098,Free Kick,1633,2H,2829.821084,30,251596232,"{'name': 'Angola', 'id': '24', 'alpha3code': '...",67.0,Jo\u00e3o M\u00e1rio,,Naval da Costa Eduardo,3161,1993-01-19,173.0,"{'code2': 'MD', 'code3': 'MID', 'name': 'Midfi...","{'name': 'Portugal', 'id': '620', 'alpha3code'...",70965.0,right,Jo\u00e3o M\u00e1rio,9905
643147,1,Air duel,"[{'id': 701}, {'id': 1802}]",7919,"[{'y': 53, 'x': 12}, {'y': 50, 'x': 14}]",2500098,Duel,1623,2H,2831.211419,10,251596410,"{'name': 'England', 'id': 0, 'alpha3code': 'XE...",82.0,Michael,,Keane,1623,1993-01-11,188.0,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'England', 'id': 0, 'alpha3code': 'XE...",7919.0,right,M. Keane,2413
643148,1,Air duel,"[{'id': 703}, {'id': 1801}]",8005,"[{'y': 47, 'x': 88}, {'y': 50, 'x': 86}]",2500098,Duel,1633,2H,2832.434399,10,251596234,"{'name': 'Wales', 'id': 0, 'alpha3code': 'XWA'...",90.0,James Michael,,Collins,null,1983-08-23,193.0,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'Wales', 'id': 0, 'alpha3code': 'XWA'...",8005.0,right,J. Collins,null


In [11]:
#Those playerIds from df events that dont have an entry in df players
noplayer = events_com[events_com['firstName'].isna().values]
noplayer

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,passportArea,weight,firstName,middleName,lastName,currentTeamId,birthDate,height,role,birthArea,wyId,foot,shortName,currentNationalTeamId
8,1,Air duel,"[{'id': 703}, {'id': 1801}]",0,"[{'y': 85, 'x': 61}, {'y': 80, 'x': 67}]",2499719,Duel,1609,1H,14.765321,10,177961218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,1,Ground loose ball duel,"[{'id': 701}, {'id': 1802}]",0,"[{'y': 78, 'x': 33}, {'y': 85, 'x': 29}]",2499719,Duel,1609,1H,38.191874,13,177961219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,3,Free Kick,[{'id': 1801}],0,"[{'y': 44, 'x': 65}, {'y': 31, 'x': 65}]",2499719,Free Kick,1609,1H,74.737141,31,177959198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,1,Ground defending duel,"[{'id': 502}, {'id': 701}, {'id': 1802}]",0,"[{'y': 86, 'x': 4}, {'y': 97, 'x': 10}]",2499719,Duel,1631,1H,85.074043,12,177961220,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,1,Ground attacking duel,"[{'id': 701}, {'id': 1802}]",0,"[{'y': 27, 'x': 89}, {'y': 27, 'x': 95}]",2499719,Duel,1631,1H,152.127886,11,177961221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
643080,5,Ball out of the field,[],0,"[{'y': 100, 'x': 51}, {'y': 100, 'x': 100}]",2500098,Interruption,1623,2H,2589.146520,50,251596374,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
643101,3,Free Kick,[{'id': 1801}],0,"[{'y': 87, 'x': 40}, {'y': 80, 'x': 56}]",2500098,Free Kick,1623,2H,2665.580593,31,251596391,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
643117,1,Ground defending duel,"[{'id': 502}, {'id': 701}, {'id': 1802}]",0,"[{'y': 34, 'x': 7}, {'y': 28, 'x': 5}]",2500098,Duel,1623,2H,2712.941293,12,251596987,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
643140,3,Free Kick,[{'id': 1801}],0,"[{'y': 81, 'x': 70}, {'y': 70, 'x': 59}]",2500098,Free Kick,1633,2H,2780.300522,31,251596224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
noplayer.subEventName.value_counts()

Ball out of the field     27326
Ground defending duel      6206
Ground attacking duel      4841
Ground loose ball duel     4096
Air duel                   2972
Free Kick                  1550
Throw in                    652
Whistle                     201
Foul                         59
Goal kick                    46
Touch                        40
Corner                       28
Simple pass                   3
                              3
Launch                        3
Reflexes                      1
Clearance                     1
Free kick cross               1
Shot                          1
Hand foul                     1
Name: subEventName, dtype: int64

These entries are the interruptions that dont need a player (mostly, rest are data collection anomalies)

In [13]:
events_com = events_com.dropna()
events_com = events_com.reset_index(drop=True)

# Extracting Player Position from Match Lineup 

## Extracting Defender Position 

In [14]:
defence = pd.read_csv('../data/matches/PLopensource.csv')

In [15]:
defence['home_team'].unique()

array(['Arsenal', 'Brighton and Hove Albion', 'Chelsea', 'Crystal Palace',
       'Everton', 'Manchester United', 'Newcastle United', 'Southampton',
       'Watford', 'West Bromwich Albion', 'AFC Bournemouth', 'Burnley',
       'Huddersfield Town', 'Leicester City', 'Liverpool',
       'Manchester City', 'Stoke City', 'Swansea City',
       'Tottenham Hotspur', 'West Ham United'], dtype=object)

In [16]:
defence.drop(columns = ['Unnamed: 0'], inplace=True)

In [17]:
def clean_tokenize(s):
    s = s.replace(",", ' ')
    s = s.replace("[", '')
    s = s.replace("]", '')
    s = s.replace("'", '')
    s = s.split()
    return s

In [18]:
defence['home_team_defense'] = defence['home_team_defense'].apply(lambda x: clean_tokenize(x))
defence['away_team_defense'] = defence['away_team_defense'].apply(lambda x: clean_tokenize(x))

In [19]:
def add_home_posn(df):
    for i in range(0, len(df)):
        if(len(df['home_team_defense'][i])==4):
            df['home_RB'][i] = df['home_team_defense'][i][0]
            df['home_R-CB'][i] = df['home_team_defense'][i][1]
            df['home_L-CB'][i] = df['home_team_defense'][i][2]
            df['home_LB'][i] = df['home_team_defense'][i][3]
            df['home_backline'][i] = 4
        elif(len(df['home_team_defense'][i])==3):
            df['home_RCB'][i] = df['home_team_defense'][i][0]
            df['home_CB'][i] = df['home_team_defense'][i][1]
            df['home_LCB'][i] = df['home_team_defense'][i][2]
            df['home_backline'][i] = 3
        else:
            df['home_RWB'][i] = df['home_team_defense'][i][0]
            df['home_RCB'][i] = df['home_team_defense'][i][1]
            df['home_CB'][i] = df['home_team_defense'][i][2]
            df['home_LCB'][i] = df['home_team_defense'][i][3]
            df['home_LWB'][i] = df['home_team_defense'][i][4]
            df['home_backline'][i] = 5

In [20]:
def add_away_posn(df):
    for i in range(0, len(df)):
        if(len(df['away_team_defense'][i])==4):
            df['away_RB'][i] = df['away_team_defense'][i][0]
            df['away_R-CB'][i] = df['away_team_defense'][i][1]
            df['away_L-CB'][i] = df['away_team_defense'][i][2]
            df['away_LB'][i] = df['away_team_defense'][i][3]
            df['away_backline'][i] = 4
        elif(len(df['away_team_defense'][i])==3):
            df['away_RCB'][i] = df['away_team_defense'][i][0]
            df['away_CB'][i] = df['away_team_defense'][i][1]
            df['away_LCB'][i] = df['away_team_defense'][i][2]
            df['away_backline'][i] = 3
        else:
            df['away_RWB'][i] = df['away_team_defense'][i][0]
            df['away_RCB'][i] = df['away_team_defense'][i][1]
            df['away_CB'][i] = df['away_team_defense'][i][2]
            df['away_LCB'][i] = df['away_team_defense'][i][3]
            df['away_LWB'][i] = df['away_team_defense'][i][4]
            df['away_backline'][i] = 5

In [21]:
defence = defence.reindex(columns=defence.columns.tolist() + [
    'home_RB',
    'home_R-CB',
    'home_L-CB',
    'home_LB',
    'home_RCB',
    'home_CB',
    'home_LCB',
    'home_RWB',
    'home_LWB',
    'away_RB',
    'away_R-CB',
    'away_L-CB',
    'away_LB',
    'away_RCB',
    'away_CB',
    'away_LCB',
    'away_RWB',
    'away_LWB',
    'home_backline',
    'away_backline'
])

In [22]:
add_home_posn(defence)
add_away_posn(defence)

In [23]:
# defence

## Merge Match_IDs 

In [24]:
def label_splitter(df):
    df['label'] = df['label'].apply(lambda x: x.replace(",", ' '))
    df['label'] = df['label'].apply(lambda x: x.split('  '))
    df['score'] = df['label'].apply(lambda x: x[1])
    df['label'] = df['label'].apply(lambda x: x[0].split(' - '))
    df['home_team'] = df['label'].apply(lambda x: x[0])
    df['away_team'] = df['label'].apply(lambda x: x[1])
    return df

In [25]:
label_splitter(matches)

,status,roundId,gameweek,teamsData,seasonId,dateutc,winner,venue,wyId,label,date,referees,duration,competitionId,score,home_team,away_team
0,Played,4405654,38,"{'1646': {'scoreET': 0, 'coachId': 8880, 'side...",181150,2018-05-13 14:00:00,1659,Turf Moor,2500089,"[Burnley, AFC Bournemouth]","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 385705, 'role': 'referee'}, {'r...",Regular,364,1 - 2,Burnley,AFC Bournemouth
1,Played,4405654,38,"{'1628': {'scoreET': 0, 'coachId': 8357, 'side...",181150,2018-05-13 14:00:00,1628,Selhurst Park,2500090,"[Crystal Palace, West Bromwich Albion]","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 381851, 'role': 'referee'}, {'r...",Regular,364,2 - 0,Crystal Palace,West Bromwich Albion
2,Played,4405654,38,"{'1609': {'scoreET': 0, 'coachId': 7845, 'side...",181150,2018-05-13 14:00:00,1609,The John Smith's Stadium,2500091,"[Huddersfield Town, Arsenal]","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 384965, 'role': 'referee'}, {'r...",Regular,364,0 - 1,Huddersfield Town,Arsenal
3,Played,4405654,38,"{'1651': {'scoreET': 0, 'coachId': 8093, 'side...",181150,2018-05-13 14:00:00,1612,Anfield,2500092,"[Liverpool, Brighton & Hove Albion]","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 385704, 'role': 'referee'}, {'r...",Regular,364,4 - 0,Liverpool,Brighton & Hove Albion
4,Played,4405654,38,"{'1644': {'scoreET': 0, 'coachId': 93112, 'sid...",181150,2018-05-13 14:00:00,1611,Old Trafford,2500093,"[Manchester United, Watford]","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 381853, 'role': 'referee'}, {'r...",Regular,364,1 - 0,Manchester United,Watford
5,Played,4405654,38,"{'1613': {'scoreET': 0, 'coachId': 210700, 'si...",181150,2018-05-13 14:00:00,1613,St. James' Park,2500094,"[Newcastle United, Chelsea]","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 384888, 'role': 'referee'}, {'r...",Regular,364,3 - 0,Newcastle United,Chelsea
6,Played,4405654,38,"{'1625': {'scoreET': 0, 'coachId': 267136, 'si...",181150,2018-05-13 14:00:00,1625,St. Mary's Stadium,2500095,"[Southampton, Manchester City]","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 385911, 'role': 'referee'}, {'r...",Regular,364,0 - 1,Southampton,Manchester City
7,Played,4405654,38,"{'10531': {'scoreET': 0, 'coachId': 32573, 'si...",181150,2018-05-13 14:00:00,1639,Liberty Stadium,2500096,"[Swansea City, Stoke City]","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 378952, 'role': 'referee'}, {'r...",Regular,364,1 - 2,Swansea City,Stoke City
8,Played,4405654,38,"{'1631': {'scoreET': 0, 'coachId': 209010, 'si...",181150,2018-05-13 14:00:00,1624,Wembley Stadium,2500097,"[Tottenham Hotspur, Leicester City]","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 378951, 'role': 'referee'}, {'r...",Regular,364,5 - 4,Tottenham Hotspur,Leicester City
9,Played,4405654,38,"{'1623': {'scoreET': 0, 'coachId': 8541, 'side...",181150,2018-05-13 14:00:00,1633,London Stadium,2500098,"[West Ham United, Everton]","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 408156, 'role': 'referee'}, {'r...",Regular,364,3 - 1,West Ham United,Everton


In [26]:
matches = matches.replace({'home_team':{
        'Brighton & Hove Albion': 'Brighton and Hove Albion',
    }, 'away_team': {
        'Brighton & Hove Albion': 'Brighton and Hove Albion',
    }}
)

In [27]:
matches['home_team'].unique()

array(['Burnley', 'Crystal Palace', 'Huddersfield Town', 'Liverpool',
       'Manchester United', 'Newcastle United', 'Southampton',
       'Swansea City', 'Tottenham Hotspur', 'West Ham United',
       'Manchester City', 'Leicester City', 'Chelsea', 'Arsenal',
       'Everton', 'AFC Bournemouth', 'Watford', 'West Bromwich Albion',
       'Stoke City', 'Brighton and Hove Albion'], dtype=object)

In [28]:
matches.drop(columns=['label'], inplace=True)
matches['match'] = matches['home_team'] + '-' + matches['away_team']

In [29]:
defence['match'] = defence['home_team'] + '-' + defence['away_team']

In [30]:
match_def = pd.merge(right=matches, left=defence, right_on='match', left_on='match')

In [31]:
match_def.columns

Index(['match_id', 'home_team_x', 'away_team_x', 'home_team_defense',
       'away_team_defense', 'home_RB', 'home_R-CB', 'home_L-CB', 'home_LB',
       'home_RCB', 'home_CB', 'home_LCB', 'home_RWB', 'home_LWB', 'away_RB',
       'away_R-CB', 'away_L-CB', 'away_LB', 'away_RCB', 'away_CB', 'away_LCB',
       'away_RWB', 'away_LWB', 'home_backline', 'away_backline', 'match',
       'status', 'roundId', 'gameweek', 'teamsData', 'seasonId', 'dateutc',
       'winner', 'venue', 'wyId', 'date', 'referees', 'duration',
       'competitionId', 'score', 'home_team_y', 'away_team_y'],
      dtype='object')

In [32]:
match_def.drop(columns=[
    'status', 'roundId', 'seasonId', 'date', 'winner',
    'match_id', 'duration', 'competitionId', 'home_team_y', 'away_team_y'
], inplace=True)

In [33]:
len(match_def)

380

In [34]:
match_def.to_pickle('../data/matches/matches_defence.pkl')

# Event Data Pre-Processing

## Normalizing Names 

In [35]:
events_com['firstName'] = events_com['firstName'].swifter.set_npartitions(
    8).apply(lambda x: x.encode().decode('unicode_escape').replace('\xad',''))
events_com['lastName'] = events_com['lastName'].swifter.set_npartitions(
    8).apply(lambda x: x.encode().decode('unicode_escape').replace('\xad',''))

In [36]:
players['firstName'] = players['firstName'].swifter.set_npartitions(
    8).apply(lambda x: x.encode().decode('unicode_escape').replace('\xad',''))
players['lastName'] = players['lastName'].swifter.set_npartitions(
    8).apply(lambda x: x.encode().decode('unicode_escape').replace('\xad',''))
players['playerName'] = players['firstName']+players['lastName']
players['playerName'] = players['playerName'].apply(lambda x: unidecode(x))

In [37]:
players['playerName'] = players['playerName'].apply(lambda x: x.replace('-', ''))

In [38]:
players['playerName'] = players['playerName'].apply(lambda x: x.replace(' ', ''))

In [39]:
players.to_pickle('../data/players/players.pkl')

## Modifying match_def 

In [40]:
match_def.columns

Index(['home_team_x', 'away_team_x', 'home_team_defense', 'away_team_defense',
       'home_RB', 'home_R-CB', 'home_L-CB', 'home_LB', 'home_RCB', 'home_CB',
       'home_LCB', 'home_RWB', 'home_LWB', 'away_RB', 'away_R-CB', 'away_L-CB',
       'away_LB', 'away_RCB', 'away_CB', 'away_LCB', 'away_RWB', 'away_LWB',
       'home_backline', 'away_backline', 'match', 'gameweek', 'teamsData',
       'dateutc', 'venue', 'wyId', 'referees', 'score'],
      dtype='object')

In [41]:
match_def_away = match_def[['wyId', 'away_team_x', 'away_team_defense', 'away_RB', 'away_R-CB', 'away_L-CB', 'away_LB', 'away_RCB',
       'away_CB', 'away_LCB', 'away_RWB', 'away_LWB', 'away_backline', 'match', 'gameweek', 'teamsData',
       'dateutc', 'venue', 'referees', 'score']]

In [42]:
match_def_away.columns = ['wyId', 'team', 'team_defense', 'RB', 'R-CB', 'L-CB', 'LB', 'RCB',
       'CB', 'LCB', 'RWB', 'LWB', 'backline', 'match', 'gameweek', 'teamsData',
       'dateutc', 'venue', 'referees', 'score']

In [43]:
match_def = match_def[['wyId', 'home_team_x', 'home_team_defense', 'home_RB', 'home_R-CB', 'home_L-CB', 'home_LB', 'home_RCB',
       'home_CB', 'home_LCB', 'home_RWB', 'home_LWB', 'home_backline', 'match', 'gameweek', 'teamsData',
       'dateutc', 'venue', 'referees', 'score']]

In [44]:
match_def.columns = ['wyId', 'team', 'team_defense', 'RB', 'R-CB', 'L-CB', 'LB', 'RCB',
       'CB', 'LCB', 'RWB', 'LWB', 'backline', 'match', 'gameweek', 'teamsData',
       'dateutc', 'venue', 'referees', 'score']

In [45]:
match_def = match_def.append(match_def_away, ignore_index=True)

In [46]:
match_def.sort_values(by=['gameweek'], inplace=True, ascending=False)

In [47]:
match_def.reset_index(drop=True)

,wyId,team,team_defense,RB,R-CB,L-CB,LB,RCB,CB,LCB,RWB,LWB,backline,match,gameweek,teamsData,dateutc,venue,referees,score
0,2500098,Everton,"[MichaelKeane, PhilJagielka, RamiroFunesMori]",NaN,NaN,NaN,NaN,MichaelKeane,PhilJagielka,RamiroFunesMori,NaN,NaN,3.0,West Ham United-Everton,38,"{'1623': {'scoreET': 0, 'coachId': 8541, 'side...",2018-05-13 14:00:00,London Stadium,"[{'refereeId': 408156, 'role': 'referee'}, {'r...",3 - 1
1,2500090,Crystal Palace,"[AaronWanBissaka, JamesTomkins, MamadouSakho, ...",AaronWanBissaka,JamesTomkins,MamadouSakho,PatrickvanAanholt,NaN,NaN,NaN,NaN,NaN,4.0,Crystal Palace-West Bromwich Albion,38,"{'1628': {'scoreET': 0, 'coachId': 8357, 'side...",2018-05-13 14:00:00,Selhurst Park,"[{'refereeId': 381851, 'role': 'referee'}, {'r...",2 - 0
2,2500089,AFC Bournemouth,"[SteveCook, NathanAke, TyroneMings]",NaN,NaN,NaN,NaN,SteveCook,NathanAke,TyroneMings,NaN,NaN,3.0,Burnley-AFC Bournemouth,38,"{'1646': {'scoreET': 0, 'coachId': 8880, 'side...",2018-05-13 14:00:00,Turf Moor,"[{'refereeId': 385705, 'role': 'referee'}, {'r...",1 - 2
3,2500098,West Ham United,"[DeclanRice, AngeloOgbonna, AaronCresswell]",NaN,NaN,NaN,NaN,DeclanRice,AngeloOgbonna,AaronCresswell,NaN,NaN,3.0,West Ham United-Everton,38,"{'1623': {'scoreET': 0, 'coachId': 8541, 'side...",2018-05-13 14:00:00,London Stadium,"[{'refereeId': 408156, 'role': 'referee'}, {'r...",3 - 1
4,2500097,Tottenham Hotspur,"[KyleWalkerPeters, TobyAlderweireld, EricDier,...",KyleWalkerPeters,TobyAlderweireld,EricDier,DannyRose,NaN,NaN,NaN,NaN,NaN,4.0,Tottenham Hotspur-Leicester City,38,"{'1631': {'scoreET': 0, 'coachId': 209010, 'si...",2018-05-13 14:00:00,Wembley Stadium,"[{'refereeId': 378951, 'role': 'referee'}, {'r...",5 - 4
5,2500096,Swansea City,"[AngelRangel, MikevanderHoorn, AlfieMawson, Ma...",AngelRangel,MikevanderHoorn,AlfieMawson,MartinOlsson,NaN,NaN,NaN,NaN,NaN,4.0,Swansea City-Stoke City,38,"{'10531': {'scoreET': 0, 'coachId': 32573, 'si...",2018-05-13 14:00:00,Liberty Stadium,"[{'refereeId': 378952, 'role': 'referee'}, {'r...",1 - 2
6,2500095,Southampton,"[CedricSoares, JackStephens, MayaYoshida, Wesl...",NaN,NaN,NaN,NaN,JackStephens,MayaYoshida,WesleyHoedt,CedricSoares,RyanBertrand,5.0,Southampton-Manchester City,38,"{'1625': {'scoreET': 0, 'coachId': 267136, 'si...",2018-05-13 14:00:00,St. Mary's Stadium,"[{'refereeId': 385911, 'role': 'referee'}, {'r...",0 - 1
7,2500094,Newcastle United,"[DeAndreYedlin, JamaalLascelles, FlorianLejeun...",DeAndreYedlin,JamaalLascelles,FlorianLejeune,PaulDummett,NaN,NaN,NaN,NaN,NaN,4.0,Newcastle United-Chelsea,38,"{'1613': {'scoreET': 0, 'coachId': 210700, 'si...",2018-05-13 14:00:00,St. James' Park,"[{'refereeId': 384888, 'role': 'referee'}, {'r...",3 - 0
8,2500093,Manchester United,"[MatteoDarmian, EricBailly, MarcosRojo, Ashley...",MatteoDarmian,EricBailly,MarcosRojo,AshleyYoung,NaN,NaN,NaN,NaN,NaN,4.0,Manchester United-Watford,38,"{'1644': {'scoreET': 0, 'coachId': 93112, 'sid...",2018-05-13 14:00:00,Old Trafford,"[{'refereeId': 381853, 'role': 'referee'}, {'r...",1 - 0
9,2500092,Liverpool,"[TrentAlexanderArnold, DejanLovren, VirgilvanD...",TrentAlexanderArnold,DejanLovren,VirgilvanDijk,AndrewRobertson,NaN,NaN,NaN,NaN,NaN,4.0,Liverpool-Brighton and Hove Albion,38,"{'1651': {'scoreET': 0, 'coachId': 8093, 'side...",2018-05-13 14:00:00,Anfield,"[{'refereeId': 385704, 'role': 'referee'}, {'r...",4 - 0


In [48]:
match_def.to_pickle('../data/matches/match+def_lineup.pkl')

## Adding Footedness 

In [49]:
match_def = pd.read_pickle('../data/matches/match+def_lineup.pkl')

In [50]:
a = match_def[['team_defense', 'backline']].explode('team_defense')

In [51]:
players = pd.read_pickle('../data/players/players.pkl')

In [52]:
p = players[['playerName', 'foot']]

In [53]:
b = pd.merge(right=p, left=a, right_on='playerName', left_on='team_defense', how='left')

In [54]:
b[b['foot'].isna()]['team_defense'].unique()

array(['RamiroFunesMori', 'AngeloOgbonna', 'AngelRangel', 'CedricSoares',
       'JamaalLascelles', 'EricBailly', 'MarcosRojo', 'Zanka',
       'AhmedHegazi', 'HectorBellerin', 'EzequielSchelotto', 'GaetanBong',
       'Danilo', 'KurtZouma', 'CesarAzpilicueta', 'DavinsonSanchez',
       'NicolasOtamendi', 'Bruno', 'JavierManquillo', 'TommySmith',
       'JosephGomez', 'AlbertoMoreno', 'LuisAntonioValencia',
       'VictorLindelof', 'NachoMonreal', 'JoelMatip', 'MiguelBritos',
       'JamesCollins', 'CucoMartina', 'DavidLuiz', 'ChancelMbemba',
       'PabloZabaleta', 'KikoFemenia', 'JoseFonte', 'JesusGamez'],
      dtype=object)

In [55]:
b.loc[b['team_defense'].isin([
    'CesarAzpilicueta',
    'HectorBellerin',
    'EricBailly',
    'KurtZouma',
    'Danilo',
    'EzequielSchelotto',
    'AhmedHegazi',
    'JamaalLascelles',
    'AngelRangel',
    'Zanka',
    'CedricSoares',
    'DavinsonSanchez', 
    'JavierManquillo',
    'TommySmith',
    'NicolasOtamendi',
    'JosephGomez',
    'LuisAntonioValencia',
    'VictorLindelof',
    'JoelMatip',
    'JamesCollins',
    'DavidLuiz',
    'CucoMartina',
    'JoseFonte',
    'ChancelMbemba',
    'PabloZabaleta',
    'KikoFemenia',
    'Bruno',
    'JesusGamez'
]), 'foot'] ='right'

In [58]:
b.loc[b['team_defense'].isin([
    'MarcosRojo',
    'RamiroFunesMori',
    'GaetanBong',
    'AngeloOgbonna', 
    'AlbertoMoreno',
    'NachoMonreal',
    'MiguelBritos'    
]), 'foot'] = 'left'

In [59]:
b['foot'].isna().sum()

0

In [60]:
b.index = a.index

In [61]:
f = [list(b['foot'][i]) for i in a.index.unique()]
f = pd.Series(f)
match_def['footedness'] = f.values
match_def['footedness'] = match_def['footedness'].apply(lambda x: '-'.join(x))

In [62]:
match_def.reset_index(drop=True, inplace=True)
# match_def.rename(columns={'wyId': 'matchId'})

In [63]:
match_def.to_pickle('../data/matches/match+def_lineup+footedness_ver2.pkl')

In [64]:
events_com['playerName'] = events_com['firstName']+events_com['lastName']
events_com.drop(columns=[
    'currentTeamId', 'passportArea', 'weight', 'firstName', 'lastName',
    'middleName', 'birthDate', 'height', 'role', 'birthArea', 'shortName',
    'currentNationalTeamId'
], inplace=True)
# events_com.rename(columns={'id': 'eventId', 'wyId': 'playerId'})

In [65]:
events_com.to_pickle('../data/events/events+player.pkl')

## Mapping Event Tags

In [66]:
event = pd.read_pickle('../data/events/events+player.pkl')

In [68]:
tag_key = pd.read_csv('../data/events/tags2name.csv')

In [69]:
k = list(zip(tag_key['Tag'], tag_key['Description']))

In [70]:
def unpack(x):
    x = [list(x[i].values()) for i in range(0, len(x))]
    return x

In [71]:
def functag2name(x):
    tag_names = list()
    for val in x:
        val = int(val)
        val_pair = [item for item in k if item[0]==val]
        tag_names.append(val_pair[0][1])
    return tag_names

In [72]:
event['tags'] = event['tags'].swifter.set_npartitions(8).apply(lambda x: unpack(x))
event['tags'] = event['tags'].swifter.set_npartitions(8).apply(lambda x: list(chain(*x)))

In [73]:
event['tags'] = event['tags'].swifter.set_npartitions(8).apply(lambda x: functag2name(x))

In [74]:
event.to_pickle('../data/events/events+players.pkl')

## Converting Positions Co-ordinates 

In [75]:
events = pd.read_pickle('../data/events/events+players.pkl')

In [76]:
def clean_coordinates(x):
    x = [list(d.values()) for d in x]
    x = [l[::-1] for l in x]
    return x

In [77]:
events['positions'] = events['positions'].swifter.set_npartitions(8).apply(lambda x: clean_coordinates(x))

In [78]:
events['positions'] = events['positions'].swifter.set_npartitions(8).apply(lambda x: [np.multiply([-1.04, 0.68], i) for i in x])
events['positions'] = events['positions'].swifter.set_npartitions(8).apply(lambda x: [np.subtract([0, 68], i) for i in x])

In [79]:
events['playerName'] = events["playerName"].apply(lambda x: unidecode(x))
events['playerName'] = events["playerName"].apply(lambda x: x.replace(' ',''))
events['playerName'] = events["playerName"].apply(lambda x: x.replace('-',''))

In [80]:
def roundcoords(x):
    final_roundedcoords = list()
    for l in x:
        roundedcoords = [np.round(val,decimals=2) for val in l]
        final_roundedcoords.append(roundedcoords)
    return final_roundedcoords

In [81]:
events['positions'] = events['positions'].apply(lambda x: roundcoords(x))

In [82]:
df_players = pd.read_pickle('../data/players/players.pkl')

In [83]:
# Adding Roles
roles_temp = df_players['role'].values

roles = list()
for i in roles_temp:
    roles.append(i['code3'])
    

players_roles = list(zip(roles,df_players['wyId'],df_players['playerName']))

df_players_roles = pd.DataFrame(players_roles,columns = ['role','playerId','playerName1'])

df_events_roles = events.merge(df_players_roles, left_on = 'playerId', right_on = 'playerId'  )

df_events_roles.drop(['playerName1'], axis = 1, inplace = True)

df_events_roles.to_pickle('../data/events/events_v2.pkl')

# Result

1. Use **events_plot-a-match.pkl** to plot match events from same match simultaneously<br><br>
2. Use **events_v2.pkl** to plot match events from different matches cumulatively over a season - used in all the analysis<br><br>
3. Use **match+def_lineup+footedness_ver2.pkl** for Defensive lineups of teams in each match from the PL 17-18 season<br><br>
**Note:** All event coordinate data has origin on the bottom left corner of the pitch when viewed horizontally and x = [0,104], y= [0,68]